# Compare ASKAP & ATCA spectrum

- need to check if bot velocity frames have the baricentric correction
- need to save tha ASKAP spectra properly converted to minus tau hi

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.io import fits
from spectral_cube import SpectralCube, BooleanArrayMask
from astropy.convolution import Gaussian1DKernel, convolve
import aplpy  
from astropy.wcs import WCS
from astropy.time import Time
import pandas as pd
import os


mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = '16'
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
%matplotlib inline

# start with ATCA cubes
- this method does not work very well, because the ATCA coordinate is not alwas exactely the same as the ASKAP one, this is probably because the cell size used is 10 arcsec for hydra, so there is a bit of error in the last two digits of dec

In [5]:
files = os.listdir('/Users/Denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Hydra/')

def make_plot(file):    
    cube_atca = SpectralCube.read('/Users/Denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Hydra/{}'.format(file))  # Open the FITS file for reading

    atca = fits.open('/Users/Denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Hydra/{}'.format(file))  # Open the FITS file for reading
    d_atca = atca[0].data
    h_atca = atca[0].header
    w_atca = WCS(h_atca, atca)
    
    c_atca = SkyCoord(ra=h_atca['CRVAL1'], dec=h_atca['CRVAL2'], unit=(u.deg, u.deg))
    pixels_atca = c_atca.to_pixel(w_atca)
    print(c_atca)

    askap_name = '{}{}'.format(c_atca.ra.to_string(unit=u.hour, sep=':').replace(':',''),c_atca.dec.to_string(unit=u.deg, sep=':')[:9].replace(':',''))
    print(askap_name)
    data_askap = pd.read_csv('./spectra/Hydra/{}_askap_spectrum.txt'.format(askap_name), sep='\t')

    # ATCA spectrum
    spectrum_atca = cube_atca[:, int(pixels_atca[1]), int(pixels_atca[0])]  
    vel_atca = cube_atca.spectral_axis
    y1 = spectrum_atca[:300]
    x1 = vel_atca[-300:].value/1000
    y2 = spectrum_atca[:300]
    x2 = vel_atca[-300:].value/1000
    x = np.concatenate((x1,x2), axis=None)
    y = np.concatenate((y1,y2), axis=None)
    pars = np.polyfit(vel_atca.value, spectrum_atca.value, 1)
    p = np.poly1d(pars)
    print(p[0])

    g = Gaussian1DKernel(4.0)
    hismooth = convolve(spectrum_atca, g, boundary='extend')
    tauhi_smooth = hismooth/p(vel_atca.value/1000)
    tauhi = spectrum_atca/p(vel_atca.value/1000)
    tauhi = np.array(tauhi)
    tau_atca = np.log(tauhi) * -1.

    #tauhi = hismooth/p(hiv)
    minustauhi = 1. - tauhi
    minustauhi_smooth = 1. - tauhi_smooth
    rms_atca = np.sqrt(np.mean(tauhi**2))


    # ASKAP spectrum
    spectrum_askap = np.array(data_askap['Amplitude'])
    vel_askap = np.array(data_askap['velocity'])

    # plot
    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(111)
    plt.plot(vel_atca/1000, tauhi, 'C0', linewidth=3,  label='ATCA')
    #plt.plot(vel_askap, spectrum_askap+1, 'C1', linewidth=3, label='ASKAP')
    plt.plot(vel_askap, (spectrum_askap+p[0])/p[0], 'C1', linewidth=3, label='ASKAP')
    #ax.axhspan(1-rms_askap, 1+rms_askap, alpha=0.5, color='lightgrey')
    plt.title('{}'.format(file[:12]), fontsize=30)
    plt.ylabel(r'e$^{-\tau}$', fontsize=28)
    plt.xlabel("v [km/s]", fontsize=28)
    plt.xlim(-200,400)
    plt.axhline(1, color='k', linestyle='--')
    plt.legend(fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=25)
    fig.savefig('spectra_{}_ASKAP_ATCA_test.png'.format(file[:12]))

In [6]:
#make_plot(files[2])

for i in files:
    try:
        make_plot(i)
    except Exception as e:
        print(e)
        continue

<SkyCoord (ICRS): (ra, dec) in deg
    (161.59666669, -26.37138886)>
104623.2-262216
[Errno 2] No such file or directory: './spectra/Hydra/104623.2-262216_askap_spectrum.txt'
<SkyCoord (ICRS): (ra, dec) in deg
    (153.06041665, -24.86694442)>
101214.5-245200
[Errno 2] No such file or directory: './spectra/Hydra/101214.5-245200_askap_spectrum.txt'


<SkyCoord (ICRS): (ra, dec) in deg
    (152.31875, -28.66833335)>
100916.5-284006
0.046399303064335005
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)


<SkyCoord (ICRS): (ra, dec) in deg
    (159.07583333, -26.21305553)>
103618.2-261246
[Errno 2] No such file or directory: './spectra/Hydra/103618.2-261246_askap_spectrum.txt'
<SkyCoord (ICRS): (ra, dec) in deg
    (156.12333334, -30.21750002)>
102429.6-301303


0.29224794552481104
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
<SkyCoord (ICRS): (ra, dec) in deg
    (157.03749999, -26.73833332)>
102809-264418
[Errno 2] No such file or directory: './spectra/Hydra/102809-264418_askap_spectrum.txt'


<SkyCoord (ICRS): (ra, dec) in deg
    (159.80625001, -25.15777779)>
103913.5-250928
0.23501251033626064
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
<SkyCoord (ICRS): (ra, dec) in deg
    (157.03749999, -26.73833332)>
102809-264418
[Errno 2] No such file or directory: './spectra/Hydra/102809-264418_askap_spectrum.txt'


...

<SkyCoord (ICRS): (ra, dec) in deg
    (159.29291667, -26.79444446)>
103710.3-264740
0.29989918683198746
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)


<SkyCoord (ICRS): (ra, dec) in deg
    (162.67666669, -27.55249999)>
105042.4-273309
0.6745048046739666
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)


<SkyCoord (ICRS): (ra, dec) in deg
    (161.15708333, -27.205)>
104437.7-271218
0.5383294925119925
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)


<SkyCoord (ICRS): (ra, dec) in deg
    (154.78333334, -27.14888887)>
101908-270855
[Errno 2] No such file or directory: './spectra/Hydra/101908-270855_askap_spectrum.txt'
<SkyCoord (ICRS): (ra, dec) in deg
    (151.6425, -29.23500001)>
100634.2-291406
0.12858413433693944
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
<SkyCoord (ICRS): (ra, dec) in deg
    (159.42708333, -28.38444445)>
103742.5-282304


0.2794226698385291
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
<SkyCoord (ICRS): (ra, dec) in deg
    (161.41916666, -29.45722221)>
104540.6-292725
[Errno 2] No such file or directory: './spectra/Hydra/104540.6-292725_askap_spectrum.txt'


# Read ASKAP files first

In [9]:
#field = 'Hydra'

def make_plot_v2(askap_file, field):    

    askap_5sig = pd.read_csv('5_sigma_askap_sources_{}.csv'.format(field))
    ra = np.array(askap_5sig['ra_hms_cont'])
    dec = np.array(askap_5sig['dec_dms_cont'])
    flux_peak = np.array(askap_5sig['flux_peak'])
    flux_int = np.array(askap_5sig['flux_int'])
    
    atca_name = '{}{}'.format(askap_file[:6], askap_file[8:13])
    cube_atca = SpectralCube.read('/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/{}/j{}_cube_4k.fits'.format(field, atca_name))  # Open the FITS file for reading

    atca = fits.open('/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/{}/j{}_cube_4k.fits'.format(field, atca_name))  # Open the FITS file for reading
    d_atca = atca[0].data
    h_atca = atca[0].header
    w_atca = WCS(h_atca, atca)
    #print(h_atca)
    
    c_atca = SkyCoord(ra=h_atca['CRVAL1'], dec=h_atca['CRVAL2'], unit=(u.deg, u.deg))
    pixels_atca = c_atca.to_pixel(w_atca)
    #print(c_atca)

    data_askap = pd.read_csv('./spectra/{}/{}'.format(field, askap_file), sep='\t')

    # ATCA spectrum
    spectrum_atca = cube_atca[:, int(pixels_atca[1]), int(pixels_atca[0])]  
    #ATCA = EarthLocation.from_geocentric(-4750915.837, 2792906.182, -3200483.747, unit='m')
    #barycorr = c_atca.radial_velocity_correction(obstime=Time(h_atca['DATE-OBS']), location=ATCA)
    #print('barycorr', barycorr, h_atca['VOBS'])
    vel_atca = cube_atca.spectral_axis 
    y1 = spectrum_atca[:300]
    x1 = vel_atca[-300:].value/1000
    y2 = spectrum_atca[:300]
    x2 = vel_atca[-300:].value/1000
    x = np.concatenate((x1,x2), axis=None)
    y = np.concatenate((y1,y2), axis=None)
    pars = np.polyfit(vel_atca.value, spectrum_atca.value, 1)
    p = np.poly1d(pars)
    print(p[0])

    g = Gaussian1DKernel(4.0)
    hismooth = convolve(spectrum_atca, g, boundary='extend')
    tauhi_smooth = hismooth/p(vel_atca.value/1000)
    tauhi = spectrum_atca/p(vel_atca.value/1000)
    tauhi = np.array(tauhi)
    tau_atca = np.log(tauhi) * -1.

    #tauhi = hismooth/p(hiv)
    minustauhi = 1. - tauhi
    minustauhi_smooth = 1. - tauhi_smooth
    rms_atca = np.sqrt(np.mean(tauhi**2))


    # ASKAP spectrum
    spectrum_askap = np.array(data_askap['Amplitude'])
    vel_askap = np.array(data_askap['velocity'])
    f_p = 1
    
    for i in range(len(ra)):
        if askap_file[:8] == ra[i].replace(':',''):
            f_p = flux_peak[i]/1000

    # plot
    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(111)
    plt.plot(vel_atca/1000, tauhi, 'C0', linewidth=3,  label='ATCA', alpha=0.8)
    plt.plot(vel_askap, spectrum_askap, 'C1', linewidth=3, label='ASKAP', alpha=0.8)
    #plt.plot(vel_askap, (spectrum_askap+f_p)/f_p, 'C1', linewidth=3, label='ASKAP') # scale with ASKAP catalogue flux
    plt.text(200, 0.9, 'ASKAP peak flux: {}'.format(round(f_p,2)), fontsize=20)
    #ax.axhspan(1-rms_askap, 1+rms_askap, alpha=0.5, color='lightgrey')
    plt.title('{}'.format(atca_name), fontsize=30)
    plt.ylabel(r'e$^{-\tau}$', fontsize=28)
    plt.xlabel("v [km/s]", fontsize=28)
    plt.xlim(-200,400)
    #plt.ylim(0.0,1.1)
    plt.axhline(1, color='k', linestyle='--')
    plt.axvline(0, color='k', linestyle='--')
    plt.grid(color='grey', linestyle='--')
    plt.legend(fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=25)
    fig.savefig('./spectra_ASKAP_ATCA/{}/spectra_{}_ASKAP_ATCA.png'.format(field, atca_name))
    

In [10]:
field = 'Norma'
#field = 'Hydra'

askap_files = os.listdir('/Users/denes/Research/High_lat_HI/ASKAP_data/spectra/{}/'.format(field))

for i in range(len(askap_files)):    
    try:
        make_plot_v2(askap_files[i], field)

    except Exception as e:
        print(e)
        continue

[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j161347-5937_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j165646-6130_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j161243-5736_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j171655-6045_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j161259-6037_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j161939-5721_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j164204-6219_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/N

0.09083021830340018
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j170320-6053_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j162049-5823_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j160352-5653_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j170421-6012_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j155339-5857_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j171717-6042_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/

0.1225952956038696
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j161855-5910_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j163250-6009_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j165405-5931_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j171512-6207_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j163126-5835_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j164909-5730_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/H

0.06608863640182686
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j161721-5847_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j171436-5930_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j172149-5828_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j160532-6132_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j161905-5923_cube_4k.fits'


0.103037432315838
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j170418-5700_cube_4k.fits'


...

0.04285804960902698
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)


0.07886886095181704
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j164445-6135_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j170127-5703_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j170750-5651_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j171858-6239_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j165356-6247_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j155908-5823_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/

0.18572755223857537
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j162854-6152_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j170653-6239_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j164144-6040_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j163525-5837_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j164526-5747_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j163512-6029_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/

0.07622165264311508
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j155927-5734_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j163952-5713_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j164502-6002_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j171151-5654_cube_4k.fits'
0.07545023318320099
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j161107-5706_cube_4k.fits'


[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j160352-5653_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j172157-6209_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j163546-5957_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j163843-6216_cube_4k.fits'
0.30809649991385
Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j171513-5712_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/High_lat_HI/ATCA_data/data_cubes_4k/Norma/j160146-5718_cube_4k.fits'
[Errno 2] No such file or directory: '/Users/denes/Research/Hig

In [68]:
ATCA = EarthLocation.from_geocentric(-4750915.837, 2792906.182, -3200483.747, unit='m') 
ATCA

<EarthLocation (-4750915.837, 2792906.182, -3200483.747) m>

In [7]:
askap_5sig = pd.read_csv('5_sigma_askap_sources.csv')

askap_5sig.keys() 

Index(['Unnamed: 0', 'id', 'catalogue_id', 'other_sbids', 'first_sbid',
       'project_id', 'island_id', 'island_name', 'n_components', 'ra_hms_cont',
       'dec_dms_cont', 'ra_deg_cont', 'dec_deg_cont', 'freq', 'maj_axis',
       'min_axis', 'pos_ang', 'flux_int', 'flux_int_err', 'flux_peak',
       'mean_background', 'background_noise', 'max_residual', 'min_residual',
       'mean_residual', 'rms_residual', 'stddev_residual', 'x_min', 'x_max',
       'y_min', 'y_max', 'n_pix', 'solid_angle', 'beam_area', 'x_ave', 'y_ave',
       'x_cen', 'y_cen', 'x_peak', 'y_peak', 'flag_i1', 'flag_i2', 'flag_i3',
       'flag_i4', 'comment', 'quality_level', 'released_date', 'rms_abs',
       'tau_max'],
      dtype='object')

In [12]:
ra = np.array(askap_5sig['ra_hms_cont'])
dec = np.array(askap_5sig['dec_dms_cont'])
flux_peak = np.array(askap_5sig['flux_peak'])

In [13]:
flux_peak

array([ 250.805,  246.479,  697.455,  227.038,  202.523,  694.487,
        205.575,  568.857,  112.88 ,  112.849,  123.559,  117.647,
        478.062,  111.47 ,  426.476,  365.438,   54.942,  308.885,
        271.068,  291.379, 1109.796,  386.44 ,   44.937,  312.936,
        161.325,   36.722,  258.492,   26.007,  873.144,  217.951,
        263.941,  179.085,  183.046,  155.609,  812.21 ,  116.628,
         21.129,  143.358,  147.95 ,  106.828,  115.55 ,  106.423,
        738.535,  101.625,  106.391,  107.767,  720.798,   81.522,
         82.07 ,  522.06 ,  508.234,  465.812,   56.421,  407.463,
         54.567,   48.611])

In [15]:
askap_name = '100748.6-271803'

for i in range(len(ra)):
    if askap_name[:8] == ra[i].replace(':',''):
        print('ok')

ok
